# 

In [1]:
from langchain.schema import SystemMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.chat_models import ChatOpenAI
import glob
import pandas as pd
from datasets import Dataset, load_from_disk

from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

import os
from dotenv import load_dotenv
import json
import tqdm

import pinecone
from langchain.vectorstores import Pinecone

import uuid

/Users/kieranschubert/Desktop/unigpt-assist/venv_unigpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load env variables

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX= os.getenv('PINECONE_INDEX')

# Define output format

In [12]:
class JSON_output(BaseModel):
    label: str = Field(description="label")
    comment: str = Field(description="comment")

parser = PydanticOutputParser(pydantic_object=JSON_output)

# Init llm

In [13]:
import sys
sys.path.append("../scripts")

from rag import init_pinecone, init_vectorstore, init_retriever, init_retrievalqa_chain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [14]:
llm  = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4-1106-preview',
    temperature=0,
    max_tokens=512
)

In [15]:
# ReAct prompt
template = """You will be presented with a user query in french. Your task is to classify the user query into one final category.
    
    Approach this task step by step, take your time and do not skip any steps.

    1. Read the user query.
    2. Determine whether the query is related to university regulation OR university courses.
       - If the query is related to general university regulations or faculty regulations, assign a label of "reglement".
       - If the query is related to course details (eg. question about a specific course, a timetable, a professor, the contents of a course, etc.), assign a label of "cours".
    4. Output a response as JSON with keys as follows:
        - "label": allowable values are ["reglement", "cours"]
        - "comment": any suitable comment based on the classification you performed, if required.

    Input query: {query}
"""

prompt_template = PromptTemplate(input_variables=["query"], template=template)

In [16]:
eval_set = pd.read_csv("../evaluation/evaluated/k_docs_5.csv")
eval_set.head()

,question,reponse,src,rag_answer
0,Quelles sont les responsabilités du rectorat d...,Le rectorat veille à la réalisation des missio...,Statut-Version-1juillet2021.pdf,Le rectorat de l'Université de Genève est resp...
1,Comment sont désignés les membres de l'assembl...,La composition de l'assemblée de l'université ...,Statut-Version-1juillet2021.pdf,Les membres de l'assemblée de l'université son...
2,Quelles sont les conditions pour être considér...,Une personne est considérée comme étudiant ou ...,Statut-Version-1juillet2021.pdf,Pour être considéré comme étudiant ou étudiant...
3,Quels sont les critères d'élimination d'un étu...,Un étudiant ou une étudiante est éliminé(e) d'...,Statut-Version-1juillet2021.pdf,Les critères d'élimination d'un étudiant ou d'...
4,Quelles sont les conditions pour l'exmatricula...,L'exmatriculation est demandée par l'étudiant ...,Statut-Version-1juillet2021.pdf,Les conditions pour l'exmatriculation d'un étu...


In [19]:
labels = []
for query in tqdm.tqdm(["Quelles sont les responsabilités du rectorat de l'université ?", "Y a-t-il un cours du professeur François Fleuret sur le deep learning ?"]):

    prompt = prompt_template.format(query=query)
    res = llm([SystemMessage(content=prompt)])

    labels.append(parser.parse(res.content))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.05s/it]


In [20]:
labels

[JSON_output(label='reglement', comment="The query asks about the responsibilities of the university's rectorate, which pertains to university governance and regulations."),
 JSON_output(label='cours', comment='The query is asking about a specific course on deep learning taught by Professor François Fleuret.')]